# Data Exploration

#### In this notebook we will explore the dataset that we are going to train our models with. We will try to find out insides about each feature and about the relations between them

We begin by importing the libraries that we are going to need for this procedure

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys

SCRIPTS_FILEPATH = "./../scripts/"
DATA_FILEPATH = "../data/train.csv"

sys.path.append(SCRIPTS_FILEPATH)
from data_cleaner import Data_Cleaner
from proj1_helpers import *

%load_ext autoreload
%autoreload 2

#### Loading the dataset
The next step is to load our dataset. We do that using the Data_Cleaner class whose initialisation does the following : Load the dataset and creates different numpy arrays for the feature matrix, the labels and the ids. Additionaly, it creates a dictionary with the feature names as keys and their index number as value pairs. We also print the shape of each of the 3 parts of the dataset to get a graps of its dimensions.

In [ ]:
data = Data_Cleaner(DATA_FILEPATH)

In [ ]:
data.feature_names.items()

#### The dataset in a glimpse
We start by taking an overall look of the dimensions of our dataset to get an idea about it. We also check about the percentage of each event/class to see if our training set is balanced.

In [ ]:
print("Shape of feature matrix : {}".format(data.tX.shape))
print("Shape of labels column : {}".format(data.y.shape))
print("Shape of ids column : {}".format(data.ids.shape))

signal_percentage = data.y[data.y == 1 ].sum() / len(data.y) * 100
background_percentage = 100 - signal_percentage
print("Percentage of signal events in dataset : {:.1f}".format(signal_percentage))
print("Percentage of background events in dataset : {:.1f}".format(background_percentage))

#### Number of unique values, mean, histogram of values
Moving on, we want to obtain some numerical measures about these features like their mean value and standard deviation as well as the percentage of the variables that are indicated as "may be undefined". From the description of the Higgs Challenge we know that these "undefined" variables are always equal to -999. So we use the function _fill_with_NaN() from our Data_Cleaner class to replace all -999 values with NaN. By doing that, we are able to calculate and print these numbers for all defined variables and finally plot 2 graphs summarizing them.

In [ ]:
data._fill_with_NaN()

percentages = []
mean_values = []
std_values = []

for feature_name, index in data.feature_names.items():
    f_vals = data.tX[:,index]
    num_unique = len(np.unique(f_vals))
    print("{} has {} unique values\n".format(feature_name,num_unique))
    
    mean = np.nanmean(f_vals)
    std = np.nanstd(f_vals)
    percentage = np.count_nonzero(~np.isnan(f_vals))/len(f_vals)*100
    mean_values.append(mean)
    std_values.append(std)
    percentages.append(percentage)    
    
    print("Percent (%) of samples that have entry: {:.1f}".format(percentage))
    print("Mean value: {:.3f}".format(mean))
    print("Standard deviation : {:.3f}".format(std))
    print("Ratio for std/mean: {:.3f}".format(std/mean))
    plt.figure()
    plt.hist(f_vals,bins= 100)
    plt.xlabel("{} distribution".format(feature_name))
    plt.ylabel('Frequency')
    plt.savefig('../plots/Distribution for {}'.format(feature_name))
    plt.show()
    print("--------------------------------\n")

plt.figure()
plt.scatter(mean_values, std_values)  
plt.xlabel('Mean value')
plt.ylabel('Standard deviation')
plt.savefig('../plots/mean_vs_std2.png')

plt.show

plt.figure(figsize=(8,4))
plt.plot(np.linspace(1,30,num = 30),percentages)
plt.xticks(np.linspace(1,30,num = 30))
plt.xlabel('Feature')
plt.ylabel('Percentage %')
plt.grid()
plt.savefig('../plots/Percentage')
plt.show

#### Cleaning the dataset 
For the following procedures the np.NaN values pose a problem that we have to solve. We identify 2 cases for variables that are indicated as "may be undefined" :
1. Variables from the first feature DER_mass_MMC which estimates the mass $m_H$ of the Higgs boson candidate. In this case these variables can't be defined as the topology of the event is too far from the expected topology. To solve this, we fill all np.NaN values of this feature with the median value of the remaining feature's values.
2. Features related to jets. These features have some of their variables undefined because jets have not been detected in the event and so they are meaningless. To tackle this problem we fill all these undefined variables with 0.

After treating all the undefined variables we also normalize our dataset.

In [ ]:
data.fix_mass_MMC(add_impute_array = False)
data.replace_with_zero()
data.normalize()

#### Correlation of features
We visualize the correlation between the features of our dataset

In [ ]:
corr_mat = np.corrcoef(data.tX,rowvar=False)
plt.matshow(corr_mat)
plt.colorbar()
plt.savefig('../plots/Correlation')
plt.show()

#### Principal component analysis

In [ ]:
cov_mat = np.cov(data.tX.T) #calculate covariance matrix
eigval_pca, eigvec_pca = np.linalg.eig(cov_mat) #can not be orderd, but they are here

total_eigval = np.sum(eigval_pca)
percentages = [eigval/total_eigval for eigval in eigval_pca]
percentages_cumulative = np.cumsum(percentages)
plt.plot(np.arange(1,len(eigval_pca)+1),percentages_cumulative)
plt.xlim(1,len(eigval_pca))
plt.ylabel("Total variance \"explained\" ")
plt.xlabel("no. principal component")
plt.savefig('../plots/PCA')
plt.show()

Arbitrarily choose cutoff when more than $0.95$% of the cumulative variance is explained

In [ ]:
greater_095 = np.argmax(percentages_cumulative > 0.95) #stops at first true
print("{} principal components can explain more than 95% of the variance".format(greater_095+1))

#### Project onto principal components
v : (..., M, M) array 
  The normalized (unit "length") eigenvectors, 
  such that the 
        column ``v[:,i]`` is the eigenvector corresponding to the 
        eigenvalue ``w[i]``.

In [ ]:
pcas_095 = eigvec_pca[:,:greater_095]
pcas_095_other = (eigvec_pca.T[:][:greater_095]).T
np.allclose(pcas_095_other, pcas_095)
projection_mat = eigvec_pca[:,:greater_095]
projected_data = data.tX @  projection_mat
projected_data.shape